# Proyecto Enseña Chile – Análisis de Reclutamiento

Este notebook tiene como objetivo analizar datos históricos de postulantes a Enseña Chile y cruzarlos con información externa del MINEDUC para identificar universidades y carreras con mayor concentración de candidatos idóneos, ayudando así a mejorar el proceso de reclutamiento.

## Objetivos específicos:
1. Caracterizar históricamente a postulantes y seleccionados.
2. Analizar la relación universidad-carrera-selección.
3. Identificar universidades y carreras prioritarias.
4. Proponer una categorización en niveles de prioridad.

# Entrega Inicial del Repositorio

## Contexto y motivación

Enseña Chile es una fundación que busca atraer profesionales talentosos al mundo educativo para
generar un impacto real en las comunidades escolares más vulnerables del país.

No obstante, sus procesos de postulación enfrentan desafíos importantes: la tasa de aceptación suele
ser baja y los perfiles de quienes postulan parecen ser bastante heterogéneos.

Este proyecto apunta a utilizar los datos históricos de postulaciones de Enseña Chile, junto con
información universitaria disponible en fuentes públicas como el MINEDUC, para detectar patrones,
brechas y oportunidades de mejora en el reclutamiento y selección del programa. La idea es elaborar
recomendaciones estratégicas que ayuden a la Fundación a focalizar mejor sus esfuerzos, diversificar
el perfil de candidatos y aumentar la probabilidad de éxito en las postulaciones.

La audiencia principal de este trabajo está compuesta por el equipo de gestión de Enseña Chile, aunque
también busca ser un aporte para investigadores y responsables de políticas educativas interesados en
comprender con mayor claridad cómo se distribuye el talento docente potencial en el país.

## Preguntas objetivo

¿Qué características académicas y sociodemográficas se correlacionan con un mayor éxito en el proceso de selección de Enseña Chile?

¿Existen universidades o carreras que presenten consistentemente una mayor proporción de postulantes aceptados?

¿Qué regiones del país presentan menor participación o tasa de éxito en postulaciones?

¿Es posible construir un modelo predictivo que estime la probabilidad de éxito de un postulante en función de sus características iniciales?

¿Cómo se pueden utilizar estos hallazgos para focalizar los esfuerzos de reclutamiento y reducir posibles sesgos hacia ciertas instituciones de elite?

## Datos

Datos internos de Enseña Chile:
- Registros históricos de postulaciones:

    - Columnas `Generación`/`Año pech`: Año de la generación donde seleccionado iniciaría su primer año de colegio
    - Columnas `# Proceso`/`Proceso`: Procesos de postulación según fechas que se deciden cada año. De 1 a 3 anuales
     - Columna `Resumen Estado Postulación`: Si está en proceso, fuera del proceso o seleccionado.
    - Columna `Estado de la Postulación`: Depende de la etapa en que se encuentra
    - Siglas usadas en celdas:
        - DE: Día de Entrevista (Etapa de selección antigua)
        - PR: Primera Revisión (Formulario: filtro 1)
        - ET: Entrevista Telefónica (filtro 1.1)
        - EG: Entrevista Grupal (filtro 2)
        - EP: Entrevista Personal (filtro 3)








Datos externos (fuentes públicas):
- Distribución de estudiantes universitarios por institución, región y carrera.
- Información de equidad y brechas (ej. género, tipo de institución).

Datos DEMRE:
- Puntajes PAES de matrícula, por año, provistos por datos abiertos del Departamento de
Medición, Evaluación y Registro Educacional de la Universidad de Chile.



## Análisis exploratorio de los datos

# 1 Configuracion inicial

## 1.1 Bibliotecas

Este notebook se desarrolló en Google Colab, donde el almacenamiento de archivos es solo temporal, por tanto, para que no tuvieramos que subir los archivos necesarios cada vez que quisieramos correr el notebook, usamos la libreria `gdown` para automatizar la descarga de archivos desde nuestra carpeta del proyecto en Google Drive.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

In [171]:
# Configuraciones de visualizacion
pd.set_option('display.max_columns', None)

# 2 Carga y Exploración Inicial de los Datos

## 2.1 Postulantes

### 2.1.1 Crear DataFrame

El archivo que nos entregaron tiene extensión `.xls`, formato usado desde Excel 97 a Excel 2003, predecesor del actual `.xlsx` introducido en Excel 2007. <br>
Intentamos cargarlo con `read_excel()`, pero Python nos da un error. La libreria `magic` dice que es un html. Intentamos con `read_html()`, pero se pierden las últimas 6 filas.
Como el formato del archivo no nos da confianza optamos por exportar el archivo original a un `.csv` desde Excel y lo cargamos con `read_csv()`.

In [2]:
ruta_csv = Path("datasets/postulantes/Postulaciones historicas ECh.csv")
df_postulantes = pd.read_csv(ruta_csv, low_memory=False)
# low_memory=False para que pandas infiera los tipos de datos despues de leer todo el archivo
# y asi no terminar con columnas con multiples tipos de datos

el `DtypeWarning` nos avisa que en la columna 0 hay varios tipos de datos, por tanto `pandas` dejó todas esas entradas como tipo `object`

### 2.1.2 Primer vistazo

Resumen de las columnas

In [4]:
df_postulantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71634 entries, 0 to 71633
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Año Pech                    67994 non-null  object 
 1   Generación                  67967 non-null  float64
 2   # Proceso                   824 non-null    float64
 3   Proceso                     67559 non-null  float64
 4   Edad                        71094 non-null  float64
 5   Resumen Estado Postulación  71627 non-null  object 
 6   Estado de la Postulación    71628 non-null  object 
 7   Nombre                      71624 non-null  object 
 8   Apellidos                   71628 non-null  object 
 9   Universidad                 40966 non-null  object 
 10  Universidad (old)           31093 non-null  object 
 11  Carrera                     67930 non-null  object 
 12  Carrera.1                   3735 non-null   object 
 13  Otra Carrera                912

# 3 Limpieza de Datos
**Objetivo:** Eliminar duplicados, manejar valores nulos, convertir tipos de datos, y estandarizar categorías.

## 3.1 Eliminar filas con observaciones

In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(df_postulantes.tail(8))

In [11]:
filas_antes = df_postulantes.shape[0]
df_postulantes = df_postulantes.iloc[:-5]
filas_despues = df_postulantes.shape[0]
eliminadas = filas_antes - filas_despues
print(f"Se eliminaron las {eliminadas} ultimas filas.")

Se eliminaron las 5 ultimas filas.


## 3.2 Eliminar filas vacias

In [12]:
filas_antes = df_postulantes.shape[0]
filas_vacias = df_postulantes.isnull().all(axis=1)
df_postulantes = df_postulantes[~filas_vacias]
filas_despues = df_postulantes.shape[0]
eliminadas = filas_antes - filas_despues
print(f"Se eliminaron {eliminadas} filas vacias.")

Se eliminaron 1 filas vacias.


## 3.3 Eliminar duplicados

In [13]:
filas_antes = df_postulantes.shape[0]
df_postulantes.drop_duplicates(inplace=True)
filas_despues = df_postulantes.shape[0]
eliminadas = filas_antes - filas_despues
print(f"Se eliminaron {eliminadas} filas duplicadas.")

Se eliminaron 865 filas duplicadas.


## Total postulantes

In [14]:
total_postulantes = df_postulantes.shape[0]
total_postulantes

70763

## Porcentaje seleccionados

In [15]:
seleccionados = ((df_postulantes['Resumen Estado Postulación'] == 'Seleccionado')
                        & (df_postulantes['Estado de la Postulación'] == 'Acepta compromiso peCh'))
total_seleccionados = df_postulantes[seleccionados].shape[0]
total_seleccionados

1725

In [16]:
porcentaje_seleccionados = total_seleccionados / total_postulantes * 100
porcentaje_seleccionados

2.4377146248745816

## 3.4 Columnas `Año Pech` y `Generación`

### 3.4.1 Convertir a Int64

In [17]:
df_postulantes = df_postulantes.astype({'Año Pech': 'Int64', 'Generación': 'Int64'})

### 3.4.2 Datos conflictivos

In [18]:
con_annos_distintos = (df_postulantes['Año Pech'].notna()
                     & df_postulantes['Generación'].notna()
                     & (df_postulantes['Año Pech'] != df_postulantes['Generación']))

df_postulantes[con_annos_distintos]

,Año Pech,Generación,# Proceso,Proceso,Edad,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"


seba: no hay conflictos entre annos

### 3.4.3 unir columnas

In [19]:
df_postulantes['Año Pech'] = df_postulantes['Año Pech'].combine_first(df_postulantes['Generación'])

df_postulantes = df_postulantes.rename(columns={'Año Pech': 'Año'})

df_postulantes = df_postulantes.drop(columns='Generación')

df_postulantes.head(1)

,Año,# Proceso,Proceso,Edad,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
0,2021,NaN,3.0,37.0,Seleccionado,Acepta compromiso peCh,Astrid,Vega Escobedo,U. Santo Tomás (Santiago),NaN,Fonoaudiología,NaN,NaN,NaN


### 3.4.4 revisar datos extraños

In [20]:
print(np.sort(df_postulantes['Año'].unique().dropna()))

[2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
 2023 2024 2025 2030]


#### revisar filas con anno 2030

In [21]:
con_anno_2030 = (df_postulantes['Año'] == 2030).fillna(False)
df_postulantes[con_anno_2030]

,Año,# Proceso,Proceso,Edad,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
71624,2030,NaN,1.0,28.0,Seleccionado,Aceptado a EG,Valentina,Test,U. de Santiago de Chile,NaN,Ingeniería de Ejecución (todas las especialida...,NaN,NaN,NaN
71625,2030,NaN,1.0,NaN,Seleccionado,Aceptado en DE,Jennifer,Test,NaN,NaN,NaN,NaN,NaN,NaN
71626,2030,NaN,1.0,28.0,Fuera del proceso,Rechazado en DE,Tamara,Test,U. Alberto Hurtado,NaN,Ciencia Política / Licenciatura en Ciencia Pol...,NaN,NaN,NaN
71627,2030,NaN,3.0,39.0,Seleccionado,Acepta compromiso peCh,Tester,Area TI,U. Técnica Federico Santa María (Santiago),U. Andrés Bello (Santiago),Ingeniería Civil Electrónica,NaN,asas,NaN


por los nombres se infiere que fueron filas de testeo que a los encargados de la base de datos se les olvidó quitar <br>
por tanto las eliminamos

In [22]:
filas_antes = df_postulantes.shape[0]
df_postulantes = df_postulantes[~con_anno_2030]
filas_despues = df_postulantes.shape[0]
eliminadas = filas_antes - filas_despues
print(f"Se eliminaron {eliminadas} filas")

Se eliminaron 4 filas


### 3.4.5 Restar 1 para obtener año de postulación

Citando a Sebastián I.: "En el 99% de los casos el año de postulación es 1 año antes de hacer el programa"

In [23]:
df_postulantes['Año'] -= 1

## 3.5 Columnas `# Proceso` y `Proceso`

eliminarlas

In [ ]:
df_postulantes.drop(columns=['# Proceso', 'Proceso'])

## 3.6 Columna `Edad`

### 3.6.1 Convertir a `Int64` y renombar a `Edad actual`

In [27]:
df_postulantes = df_postulantes.astype({'Edad': 'Int64'})
df_postulantes = df_postulantes.rename(columns={'Edad': 'Edad actual'})

### 3.6.2 Calcular edad al postular

In [28]:
# creacion del excel fue '2025-09-16 11:57' lo simplifcamos como 2025.7 para nuestros calculos
edad_con_decimales = df_postulantes['Edad actual'] - (2025.7 - df_postulantes['Año'])
# ACLARACION: si la persona tenia la celda `Año` vacia, su edad tambien quedara vacia

# Truncar el resultado e insertar en el df
df_postulantes.insert(loc=3, column='Edad', value=edad_con_decimales.astype('Int64'))


In [29]:
df_postulantes.head()

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
0,2020,<NA>,3,31,37,Seleccionado,Acepta compromiso peCh,Astrid,Vega Escobedo,U. Santo Tomás (Santiago),NaN,Fonoaudiología,NaN,NaN,NaN
1,2020,<NA>,1,23,29,Seleccionado,Acepta compromiso peCh,Macarena Alejandra,Buschmann Cambiaso,Pontificia U. Católica de Chile,NaN,Ingeniería Comercial,NaN,NaN,NaN
2,2019,<NA>,1,24,31,Seleccionado,Acepta compromiso peCh,Martín,Jaeger,U. de Chile,NaN,Economía,NaN,NaN,NaN
3,2020,<NA>,1,24,30,Seleccionado,Acepta compromiso peCh,María Ignacia,Gibson,U. de Chile,NaN,Ingeniería Comercial,NaN,NaN,NaN
4,2020,<NA>,1,29,35,Seleccionado,Acepta compromiso peCh,Evelyn,Espinosa,U. Andrés Bello (Santiago),NaN,Bioquímica,NaN,NaN,NaN


### 3.6.2 mostrar valores unicos

In [30]:
print(np.sort(df_postulantes['Edad'].unique()))

[-953. -940.  -47.  -46.  -44.  -42.  -40.  -39.  -37.  -35.  -34.  -32.
  -31.  -30.  -29.  -27.  -22.  -16.   -6.   -5.   -3.   -2.   -1.    0.
    1.    2.    9.   12.   13.   14.   15.   16.   17.   18.   19.   20.
   21.   22.   23.   24.   25.   26.   27.   28.   29.   30.   31.   32.
   33.   34.   35.   36.   37.   38.   39.   40.   41.   42.   43.   44.
   45.   46.   47.   48.   49.   50.   51.   52.   53.   54.   55.   56.
   57.   58.   59.   60.   61.   62.   63.   64.   65.   66.   67.   68.
   69.   70.   71.   72.   73.   74.   75.   76.   77.   78.   98.  112.
  113.  123.  127.   nan]


## 3.7 Columna `Nombre`

### 3.7.1 Revisar si hay personas sin nombre

In [31]:
sin_nombre = df_postulantes["Nombre"].isnull()
df_postulantes[sin_nombre]

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
5037,2012,<NA>,2,34,48,Incompleta,Postulación incompleta,NaN,Aguayo Mesias,NaN,U. Tecnológica de Chile (INACAP),Traducción / Intérprete,NaN,NaN,NaN
5372,2012,<NA>,<NA>,24,38,Incompleta,Postulación incompleta,NaN,Ramírez Donaire,NaN,Otra,Arte Dramático / Actuación / Teatro,NaN,NaN,NaN
30059,2016,<NA>,3,23,33,Incompleta,Postulación incompleta,NaN,guzmán ortega,NaN,U. del Bío Bío,Pedagogía Educación Media en Historia / Cienci...,NaN,NaN,NaN
33386,2016,<NA>,2,27,37,Fuera del proceso,Rechazado en PR,NaN,santana villa,NaN,U. de La Frontera,Tecnología Médica,NaN,NaN,NaN


no parecen redundantes

## 3.8 Columna `Universidad` y `Universidad (old)`

### 3.8.1 Datos conflictivos

In [32]:
con_ues_distintas = (df_postulantes['Universidad'].notna()
                     & df_postulantes['Universidad (old)'].notna()
                     & (df_postulantes['Universidad'] != df_postulantes['Universidad (old)']))

display(df_postulantes[con_ues_distintas].sort_values("Universidad"))

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Universidad (old),Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
34780,2017,<NA>,11,23,32,Seleccionado,Acepta compromiso peCh,Claudia Patricia,Millafilo Antilef,Pontificia U. Católica de Chile,U. Católica de la Santísima Concepción,Pedagogía en Inglés,NaN,NaN,NaN
28454,2016,<NA>,3,24,34,Seleccionado,Acepta compromiso peCh,Paulina Andrea,Hinojosa Alarcón,Sin información,Pontificia U. Católica de Valparaíso,Sin información,NaN,NaN,NaN
3708,2009,<NA>,<NA>,<NA>,<NA>,Seleccionado,Acepta compromiso peCh,Víctor,Ruiz,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Filosofía,NaN,NaN,NaN
14959,2013,<NA>,3,23,36,Seleccionado,Acepta compromiso peCh,Javiera,García García,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Periodismo / Comunicación Social,NaN,NaN,NaN
9976,2013,<NA>,1,23,36,Seleccionado,Acepta compromiso peCh,Camila Del Rosario,López De La Barrera,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Ingeniería Comercial,NaN,NaN,NaN
15364,2014,<NA>,1,23,35,Seleccionado,Acepta compromiso peCh,Beatriz,Lyng Errázuriz,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Ingeniería Comercial/Administración/ Contabilidad,NaN,NaN,NaN
6317,2012,<NA>,<NA>,23,37,Seleccionado,Acepta compromiso peCh,Macarena Loreto,Salvo Cruces,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Periodismo / Comunicación Social,NaN,NaN,NaN
6174,2012,<NA>,<NA>,22,36,Seleccionado,Acepta compromiso peCh,TAIRA SOFÍA,CORREA SIRVENT,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Ingeniería Comercial,NaN,NaN,NaN
3709,2009,<NA>,<NA>,24,41,Seleccionado,Acepta compromiso peCh,Jimena Manola Lucia,Saavedra Fernández,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Pedagogía Educación Media en Historia / Cienci...,NaN,NaN,NaN
8877,2012,<NA>,2,21,35,En proceso de selección,Sin revisar,consuelo,leyton terrazas,U. Adolfo Ibáñez (Viña del Mar),U. Adolfo Ibáñez (Santiago),Historia / Licenciatura en Historia,NaN,NaN,NaN


### 3.8.2 Unir columnas

In [33]:
df_postulantes['Universidad'] = df_postulantes['Universidad (old)'].combine_first(df_postulantes['Universidad'])

df_postulantes = df_postulantes.drop(columns='Universidad (old)')

df_postulantes.head(1)

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
0,2020,<NA>,3,31,37,Seleccionado,Acepta compromiso peCh,Astrid,Vega Escobedo,U. Santo Tomás (Santiago),Fonoaudiología,NaN,NaN,NaN


### 3.8.3 estandarizar nombres universidades

Para poder explorar mejor todas las entradas para universidad que hay en la database, exportamos a json

In [34]:
universidades = df_postulantes['Universidad'].sort_values().unique().tolist()

#crear carpeta
Path('por mapear').mkdir(parents=True, exist_ok=True)
ruta_archivo = Path('por mapear/universidades.json')

with open(ruta_archivo, 'w', encoding='utf-8') as file:
    json.dump(universidades, file, ensure_ascii=False, indent=2)

In [35]:
universidades_estandarizadas = {
    "U. Arcis": "U. de Arte y Ciencias Sociales - ARCIS",
    "U. de Arte y Ciencias Sociales - ARCIS": "U. de Arte y Ciencias Sociales - ARCIS",
    "U. Tecnológica de Chile (INACAP)": "U. Tecnológica de Chile - INACAP",
    "U. Tecnológica de Chile - INACAP": "U. Tecnológica de Chile - INACAP",
    "U. de Ciencias de la Informática": "U. de Ciencias de la Informática - UCINF",
    "U. de Ciencias de la Informática - UCINF": "U. de Ciencias de la Informática - UCINF",
    "U. Metropolitana de Ciencias de la Educación": "U. Metropolitana de Ciencias de la Educación - UMCE",
    "U. Metropolitana de Ciencias de la Educación - UMCE": "U. Metropolitana de Ciencias de la Educación - UMCE",
    "U. UNIACC": "U. de Artes, Ciencias y Comunicación - UNIACC",
    "U. de Artes, Ciencias y Comunicación - UNIACC": "U. de Artes, Ciencias y Comunicación - UNIACC",
    "UNIACC": "U. de Artes, Ciencias y Comunicación - UNIACC",
    "U. Tecnológica Metropolitana": "U. Tecnológica Metropolitana - UTEM",
    "U. Tecnológica Metropolitana - UTEM": "U. Tecnológica Metropolitana - UTEM",
    "U. Tecnológica Metropolitana UTEM": "U. Tecnológica Metropolitana - UTEM",
    "Duoc UC": "Duoc UC",
    "DuocUC": "Duoc UC",
    "Escuela Moderna de Música": "Escuela Moderna de Música y Danza",
    "Escuela Moderna de Música y Danza": "Escuela Moderna de Música y Danza",
    "U. Austral": "U. Austral de Chile",
    "U. Austral de Chile": "U. Austral de Chile",
    "U. Central": "U. Central de Chile",
    "U. Central de Chile": "U. Central de Chile",
    "U. de Playa Ancha": "U. de Playa Ancha de Ciencias de la Educación",
    "U. de Playa Ancha de Ciencias de la Educación": "U. de Playa Ancha de Ciencias de la Educación",
    "U. de Santiago": "U. de Santiago de Chile",
    "U. de Santiago de Chile": "U. de Santiago de Chile",
    "U. Católica de La Santísima Concepción": "U. Católica de la Santísima Concepción",
    "U. Católica de la Santísima Concepción": "U. Católica de la Santísima Concepción",
    "U. de La Serena": "U. de La Serena",
    "U. de la Serena": "U. de La Serena",
    "U. de Las Américas": "U. de Las Américas",
    "U. de las Américas": "U. de Las Américas",
    "U. de Los Andes": "U. de Los Andes",
    "U. de los Andes": "U. de Los Andes",
    "U. de Los Lagos": "U. de Los Lagos",
    "U. de los Lagos": "U. de Los Lagos",
    "U. del Bío Bío": "U. del Bío-Bío",
    "U. del Bío-Bío": "U. del Bío-Bío",
    "U. de O' Higgins": "U. de O'Higgins",
    "U. de O'Higgins": "U. de O'Higgins",
    "U. Iberoamericana de Ciencias y Tecnología - UNICIT": "U. Iberoamericana de Ciencias y Tecnología - UNICIT",
    "U. Iberoamericana de Ciencias y Tecnología UNICIT": "U. Iberoamericana de Ciencias y Tecnología - UNICIT",
    "Otra": pd.NA,
    "Sin información": pd.NA
}

In [36]:
df_postulantes['Universidad'] = df_postulantes['Universidad'].replace(universidades_estandarizadas)
df_postulantes = df_postulantes.astype({'Universidad': 'category'})

### 3.8.1 Identificar zonas geográficas

crear columna `Zona` mapeando las universidades a la zona geografica correspondiente

In [ ]:
mapeo_zonas = {
    # ZONA NORTE
    'U. Arturo Prat de Iquique': 'Norte',
    'U. Católica del Norte': 'Norte',
    'U. de Antofagasta': 'Norte',
    'U. de Atacama': 'Norte',
    'U. de Tarapacá': 'Norte',
    'U. de La Serena': 'Norte',

    # ZONA CENTRO-NORTE
    'Pontificia U. Católica de Valparaíso': 'Centro-Norte',
    'U. Adolfo Ibáñez (Viña del Mar)': 'Centro-Norte',
    'U. Andrés Bello (Viña del Mar)': 'Centro-Norte',
    "U. Marítima de Chile": 'Centro-Norte',
    "U. de Playa Ancha de Ciencias de la Educación": 'Centro-Norte',
    'U. de Valparaíso': 'Centro-Norte',
    'U. de Viña del Mar': 'Centro-Norte',
    'U. del Mar': 'Centro-Norte',
    "U. San Sebastián (Viña del Mar)": 'Centro-Norte',
    'U. Santo Tomás (Viña del Mar)': 'Centro-Norte',
    'U. Técnica Federico Santa María (Valparaíso)': 'Centro-Norte',

    # ZONA METROPOLITANA
    'Pontificia U. Católica de Chile': 'Metropolitana',
    'U. Academia de Humanismo Cristiano': 'Metropolitana',
    'U. Adolfo Ibáñez (Santiago)': 'Metropolitana',
    'U. Alberto Hurtado': 'Metropolitana',
    'U. Andrés Bello (Santiago)': 'Metropolitana',
    "U. Bernardo O'Higgins": 'Metropolitana',
    "U. Bolivariana": 'Metropolitana',
    'U. Católica Cardenal Raúl Silva Henríquez': 'Metropolitana',
    'U. Central de Chile': 'Metropolitana',
    "U. Chileno Británica de Cultura": 'Metropolitana',
    "U. de Arte y Ciencias Sociales - ARCIS": 'Metropolitana',
    'U. de Artes, Ciencias y Comunicación - UNIACC': 'Metropolitana',
    'U. de Chile': 'Metropolitana',
    "U. de Ciencias de la Informática - UCINF": 'Metropolitana',
    'U. de Santiago de Chile': 'Metropolitana',
    'U. de Los Andes': 'Metropolitana',
    "U. de O'Higgins": 'Metropolitana',      #es de region ohiggins
    'U. del Desarrollo (Santiago)': 'Metropolitana',
    'U. del Pacífico': 'Metropolitana',
    'U. Diego Portales': 'Metropolitana',
    'U. Finis Terrae': 'Metropolitana',
    'U. Gabriela Mistral': 'Metropolitana',
    'U. Iberoamericana de Ciencias y Tecnología - UNICIT': 'Metropolitana',
    "U. Internacional SEK": 'Metropolitana',
    "U. La República": 'Metropolitana',
    "U. Los Leones": 'Metropolitana',
    'U. Mayor': 'Metropolitana',
    "U. Metropolitana de Ciencias de la Educación - UMCE": 'Metropolitana',
    'U. Pedro de Valdivia': 'Metropolitana',
    'U. San Sebastián (Santiago)': 'Metropolitana',
    'U. Santo Tomás (Santiago)': 'Metropolitana',
    "U. Tecnológica Metropolitana - UTEM": 'Metropolitana',
    'U. Técnica Federico Santa María (Santiago)': 'Metropolitana',

    # ZONA CENTRO-SUR
    'U. Adventista de Chile': 'Centro-Sur',
    'U. Andrés Bello (Concepción)': 'Centro-Sur',
    'U. Católica de Temuco': 'Centro-Sur',
    'U. Católica de la Santísima Concepción': 'Centro-Sur',
    'U. Católica del Maule': 'Centro-Sur',
    'U. de Concepción': 'Centro-Sur',
    'U. de La Frontera': 'Centro-Sur',
    'U. de Talca': 'Centro-Sur',
    'U. del Bío-Bío': 'Centro-Sur',
    'U. del Desarrollo (Concepción)': 'Centro-Sur',
    'U. San Sebastián (Concepción)': 'Centro-Sur',
    'U. San Sebastián (Puerto Montt)': 'Centro-Sur',
    'U. San Sebastián (Valdivia)': 'Centro-Sur',
    'U. Santo Tomás (Concepción)': 'Centro-Sur',

    # PATAGONIA
    'U. de Los Lagos': 'Patagonia',
    'U. Austral de Chile': 'Patagonia',
    'U. de Aysén': 'Patagonia',
    'U. de Magallanes': 'Patagonia',

    # NO ESPECIFICA
    'U. San Sebastián': 'No especifica',
    'U. del Desarrollo': 'No especifica',
    "Centro de Formación Técnica INACAP": 'No especifica',
    "Duoc UC": 'No especifica',
    "Escuela Moderna de Música y Danza": 'No especifica',
    "Instituto Profesional INACAP": 'No especifica',
    'U. Autónoma de Chile': 'No especifica',
    "U. Autónoma del Sur": 'No especifica',
    "U. Tecnológica de Chile - INACAP": 'No especifica',
    "U. de Aconcagua": 'No especifica',
    'U. de Las Américas': 'No especifica',
    "Docente de Biología y Ciencias para la Ciudadanía": 'No especifica'
}

In [ ]:
columna_zona = df_postulantes['Universidad'].map(mapeo_zonas)
df_postulantes.insert(8, 'Zona', columna_zona)
df_postulantes = df_postulantes.astype({'Zona': 'category'})

## 3.9 Columnas carreras


### 3.9.1 datos conflictivos

mostrar filas que tengan al menos 2 valores no-nulos en las columnas de carreras

In [37]:
with pd.option_context('display.max_colwidth', None):
    display(df_postulantes.dropna(thresh=2, subset=['Carrera',
                                              'Carrera.1',
                                              'Otra Carrera',
                                              'Otra carrera, ¿Cuál?']))

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera,Carrera.1,Otra Carrera,"Otra carrera, ¿Cuál?"
36,<NA>,<NA>,1,<NA>,30,Fuera del proceso,Sin cupo,maria paz,muñoz perez,U. Católica Cardenal Raúl Silva Henríquez,NaN,Pedagogía Educación Media en Lenguaje y Comunicación,NaN,entrenador deportivo
40,<NA>,<NA>,1,<NA>,27,Fuera del proceso,Abandona EG,Fernanda Javiera Paz,Alvear Vásquez,U. de Valparaíso,NaN,Sociología,NaN,Actualmente estudiando Trabajo Social
61,<NA>,<NA>,<NA>,<NA>,60,Fuera del proceso,No cumple requisito habilitación,César Alamiro,Cañete Gonzalez,<NA>,NaN,Ingeniería en Recursos Humanos,NaN,Contador general (Comercial)
64,<NA>,<NA>,1,<NA>,23,Fuera del proceso,Rechazado en PR,Aracelli Antonia,Orellana Charles,U. Andrés Bello (Viña del Mar),NaN,Pedagogía Educación Media en Artes Plásticas/ Tecnología,NaN,"tengo Licenciatura en Artes Visuales, y actualmente estudio; Pedagogia en Educación Media para Titulados y Licenciados."
67,<NA>,<NA>,<NA>,<NA>,40,Fuera del proceso,Rechazado en PR,María Alejandra,Ortiz Garcia,U. de Chile,NaN,Pedagogía Básica / Educación Básica,NaN,Magister en didáctica
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71616,2024,<NA>,2,29,31,Seleccionado,Acepta compromiso peCh,Kevin Ignacio,Covarrubias Alegría,U. de Playa Ancha de Ciencias de la Educación,NaN,Pedagogía Básica / Educación Básica,NaN,Pedagogía básica mención área rural y desarrollo
71620,2024,<NA>,<NA>,25,27,Seleccionado,Acepta compromiso peCh,Valentina Constanza,Guzmán Carrasco,U. de Concepción,Traducción / Intérprete,Traducción / Intérprete,NaN,NaN
71621,2024,<NA>,11,28,30,Fuera del proceso,Desvinculado por eCh,Valentina Eugenia,Muñoz Osorio,U. de Concepción,Geofísica,Geofísica,NaN,NaN
71622,2024,<NA>,11,31,33,Fuera del proceso,Desvinculado por eCh,Karina Danae,Soto Larrecheda,U. San Sebastián (Santiago),Pedagogía en Inglés / Idiomas,Pedagogía en Inglés / Idiomas,NaN,NaN


### 3.9.2 unir columnas `Carrera` y `Carrera.1`

se unen las columnas asumiendo que `Carrera.1` tiene prioridad

In [38]:
df_postulantes['Carrera'] = df_postulantes['Carrera'].combine_first(df_postulantes['Carrera.1'])
df_postulantes = df_postulantes.drop(columns=['Carrera.1'])

df_postulantes.dropna(thresh=2, subset=['Carrera', 'Otra Carrera', 'Otra carrera, ¿Cuál?']).head(1)

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera,Otra Carrera,"Otra carrera, ¿Cuál?"
36,<NA>,<NA>,1,<NA>,30,Fuera del proceso,Sin cupo,maria paz,muñoz perez,U. Católica Cardenal Raúl Silva Henríquez,Pedagogía Educación Media en Lenguaje y Comuni...,NaN,entrenador deportivo


### 3.9.3 Unir columna `Carrera` con `'Otra Carrera'` y `'Otra carrera, ¿Cuál?'`

buscar celdas en columna `Carrera` que contengan variaciones de la palabra `otro`

In [39]:
set_otras = {'otro', 'otra', 'Otro', 'Otra'}
patron_otras = '|'.join(set_otras)

puso_otra = df_postulantes['Carrera'].str.contains(patron_otras, na=False)
df_postulantes[puso_otra]

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera,Otra Carrera,"Otra carrera, ¿Cuál?"
71,<NA>,<NA>,<NA>,<NA>,25,Fuera del proceso,Rechazado en DE,Maria Jesus,Covarrubias Duran,Duoc UC,Otra,NaN,Ingeneria agricola
92,<NA>,<NA>,<NA>,<NA>,27,Fuera del proceso,No cumple requisito habilitación,Ricardo Esteban,Osses Cartes,U. Técnica Federico Santa María (Valparaíso),Otra,NaN,Ingeniería en mantenimiento industrial y Licen...
98,<NA>,<NA>,<NA>,<NA>,32,Fuera del proceso,No cumple requisito habilitación,Mauricio Hernan,Suarez Muñoz,U. de Playa Ancha de Ciencias de la Educación,Otra,NaN,Administración turística multilingüe
101,<NA>,<NA>,<NA>,<NA>,33,Fuera del proceso,No cumple requisito habilitación,Escarlet Aurora,Leal Montero,U. Católica de Temuco,Otra,NaN,Ingeniería en Prevención de Riesgos y Medio am...
108,<NA>,<NA>,<NA>,<NA>,24,Fuera del proceso,No cumple requisito habilitación,Barbara,Valenzuela,<NA>,Otra,NaN,Aiep
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71392,2023,<NA>,2,26,29,Seleccionado,Acepta compromiso peCh,Maciel Elizabeth,Müller Noriega,U. Tecnológica de Chile - INACAP,Otra Ingeniería,Ingeniería en Automatización y control industr...,NaN
71400,2023,<NA>,2,26,29,Fuera del proceso,Abandona EG,Nicolás Ignacio,Cornejo Daroch,U. Católica de la Santísima Concepción,Otra Ingeniería,Ingeniería Civil Geológica,NaN
71506,2023,<NA>,2,34,37,Fuera del proceso,No llega EG,Catherine Elizabeth,Rodríguez Aracena,Pontificia U. Católica de Valparaíso,Otra Ingeniería,Ingenieria en ejecucion de bioprocesos,NaN
71575,2024,<NA>,1,25,27,Seleccionado,Acepta compromiso peCh,Diego Andres,Quintana Ahumada,Duoc UC,Otra,NaN,Ingeniería en conectividad y redes


reemplazar esas celdas por las que están en las otras dos columnas

In [40]:
otra_carrera = df_postulantes['Otra Carrera'].fillna(df_postulantes['Otra carrera, ¿Cuál?'])

tiene_carrera = puso_otra & otra_carrera.notna()

df_postulantes.loc[tiene_carrera, 'Carrera'] = otra_carrera[tiene_carrera]
df_postulantes[puso_otra]

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera,Otra Carrera,"Otra carrera, ¿Cuál?"
71,<NA>,<NA>,<NA>,<NA>,25,Fuera del proceso,Rechazado en DE,Maria Jesus,Covarrubias Duran,Duoc UC,Ingeneria agricola,NaN,Ingeneria agricola
92,<NA>,<NA>,<NA>,<NA>,27,Fuera del proceso,No cumple requisito habilitación,Ricardo Esteban,Osses Cartes,U. Técnica Federico Santa María (Valparaíso),Ingeniería en mantenimiento industrial y Licen...,NaN,Ingeniería en mantenimiento industrial y Licen...
98,<NA>,<NA>,<NA>,<NA>,32,Fuera del proceso,No cumple requisito habilitación,Mauricio Hernan,Suarez Muñoz,U. de Playa Ancha de Ciencias de la Educación,Administración turística multilingüe,NaN,Administración turística multilingüe
101,<NA>,<NA>,<NA>,<NA>,33,Fuera del proceso,No cumple requisito habilitación,Escarlet Aurora,Leal Montero,U. Católica de Temuco,Ingeniería en Prevención de Riesgos y Medio am...,NaN,Ingeniería en Prevención de Riesgos y Medio am...
108,<NA>,<NA>,<NA>,<NA>,24,Fuera del proceso,No cumple requisito habilitación,Barbara,Valenzuela,<NA>,Aiep,NaN,Aiep
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71392,2023,<NA>,2,26,29,Seleccionado,Acepta compromiso peCh,Maciel Elizabeth,Müller Noriega,U. Tecnológica de Chile - INACAP,Ingeniería en Automatización y control industr...,Ingeniería en Automatización y control industr...,NaN
71400,2023,<NA>,2,26,29,Fuera del proceso,Abandona EG,Nicolás Ignacio,Cornejo Daroch,U. Católica de la Santísima Concepción,Ingeniería Civil Geológica,Ingeniería Civil Geológica,NaN
71506,2023,<NA>,2,34,37,Fuera del proceso,No llega EG,Catherine Elizabeth,Rodríguez Aracena,Pontificia U. Católica de Valparaíso,Ingenieria en ejecucion de bioprocesos,Ingenieria en ejecucion de bioprocesos,NaN
71575,2024,<NA>,1,25,27,Seleccionado,Acepta compromiso peCh,Diego Andres,Quintana Ahumada,Duoc UC,Ingeniería en conectividad y redes,NaN,Ingeniería en conectividad y redes


### 3.9.4 eliminar columnas `'Otra Carrera'` y `'Otra carrera, ¿Cuál?'`

In [41]:
df_postulantes = df_postulantes.drop(columns=['Otra Carrera'])
df_postulantes = df_postulantes.drop(columns=['Otra carrera, ¿Cuál?'])

In [42]:
df_postulantes.head()

,Año,# Proceso,Proceso,Edad,Edad actual,Resumen Estado Postulación,Estado de la Postulación,Nombre,Apellidos,Universidad,Carrera
0,2020,<NA>,3,31,37,Seleccionado,Acepta compromiso peCh,Astrid,Vega Escobedo,U. Santo Tomás (Santiago),Fonoaudiología
1,2020,<NA>,1,23,29,Seleccionado,Acepta compromiso peCh,Macarena Alejandra,Buschmann Cambiaso,Pontificia U. Católica de Chile,Ingeniería Comercial
2,2019,<NA>,1,24,31,Seleccionado,Acepta compromiso peCh,Martín,Jaeger,U. de Chile,Economía
3,2020,<NA>,1,24,30,Seleccionado,Acepta compromiso peCh,María Ignacia,Gibson,U. de Chile,Ingeniería Comercial
4,2020,<NA>,1,29,35,Seleccionado,Acepta compromiso peCh,Evelyn,Espinosa,U. Andrés Bello (Santiago),Bioquímica


### 3.9.5 estandarizar nombres carreras

In [43]:
#guardar carreras en json para facilitar lectura manual
carreras = df_postulantes['Carrera'].dropna().unique()
carreras.sort()     #seba: faltaba sort
carreras = carreras.tolist()

with open(Path('por mapear/carreras.json'), 'w', encoding='utf-8') as f:
    json.dump(carreras, f, ensure_ascii=False, indent=2)

Diccionario con carreras estandarizadas por Gemini 2.5 Pro

In [44]:
with open(Path("mapeos/carreras_definitivo.json"), encoding='utf-8') as f:
    carreras_estandarizadas = json.load(f)

In [45]:
#mapear diccionario
df_postulantes['Carrera'] = df_postulantes['Carrera'].map(carreras_estandarizadas)

## exportar

In [ ]:
df_postulantes